Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.


In [13]:
import json
from azureml.core import Workspace
from azureml.core.model import Model

# Cargar el archivo config.json
with open("config.json", "r") as f:
    config = json.load(f)

# Crear el workspace
ws = Workspace.create(
    name=config["workspace_name"],
    subscription_id=config["subscription_id"],
    resource_group=config["resource_group"],
    location=config["location"],
    create_resource_group=False,
    exist_ok=True
)

# Registrar el modelo
mname = "modelo"
registered_model = Model.register(
    model_path="modelo.pkl",
    model_name=mname,
    workspace=ws
)

Registering model modelo


In [8]:
################ score.py ###################
scorepy = """
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('modelo')
  model = joblib.load(model_path)

def run(raw_data):
  try:
    # Parse the input data
    data = json.loads(raw_data)['data']
    data = pd.DataFrame(data)
    
    required_columns = [
      'Title_Mr.', 'Title_Ms.', 'Title_Sr.', 'Title_Sra.',
      'SalesPerson_adventure-works\\david8', 'SalesPerson_adventure-works\\garrett1',
      'SalesPerson_adventure-works\\jae0', 'SalesPerson_adventure-works\\jillian0',
      'SalesPerson_adventure-works\\josé1', 'SalesPerson_adventure-works\\linda3',
      'SalesPerson_adventure-works\\michael9', 'SalesPerson_adventure-works\\pamela0',
      'SalesPerson_adventure-works\\shu0'
    ]
    
    # Check if we received raw or encoded data
    if 'Title' in data.columns and 'SalesPerson' in data.columns:
      # Create dummy variables - matching the training format exactly
      title_dummies = pd.get_dummies(data['Title'], prefix='Title')
      salesperson_dummies = pd.get_dummies(data['SalesPerson'], prefix='SalesPerson')
      
      # Make sure all expected columns are present
      for col in required_columns:
        prefix = col.split('_')[0]
        value = '_'.join(col.split('_')[1:])
        
        if prefix == 'Title':
          if col not in title_dummies:
            title_dummies[col] = 0
        elif prefix == 'SalesPerson':
          if col not in salesperson_dummies:
            salesperson_dummies[col] = 0
      
      # Combine the data frames
      data_encoded = pd.concat([title_dummies, salesperson_dummies], axis=1)
      
      # Only keep the columns used during training
      data_encoded = data_encoded[required_columns]
      
    elif all(col in data.columns for col in required_columns):
      # We received already encoded data
      data_encoded = data[required_columns]
    else:
      missing_cols = [col for col in required_columns if col not in data.columns]
      return json.dumps(f"Missing required columns: {', '.join(missing_cols[:5])}...")
    
    # Make the prediction
    result = model.predict(data_encoded).tolist()
    return json.dumps(result)
    
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [9]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'numpy', 'joblib']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

/var/folders/b5/6d3jsq3n12j27vfn8yqh_y500000gn/T/ipykernel_29056/381904218.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [10]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
